# Data Cleaning Appendix

In [142]:
import numpy as np
import pandas as pd
import duckdb

Load mental disorder dataframe:

In [143]:
og_df = pd.read_csv('source_disorder_data.csv')
og_df

,Entity,Code,Year,Schizophrenia disorders (share of population) - Sex: Both - Age: Age-standardized,Depressive disorders (share of population) - Sex: Both - Age: Age-standardized,Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized,Bipolar disorders (share of population) - Sex: Both - Age: Age-standardized,Eating disorders (share of population) - Sex: Both - Age: Age-standardized
0,Afghanistan,AFG,1990,0.223206,4.996118,4.713314,0.703023,0.127700
1,Afghanistan,AFG,1991,0.222454,4.989290,4.702100,0.702069,0.123256
2,Afghanistan,AFG,1992,0.221751,4.981346,4.683743,0.700792,0.118844
3,Afghanistan,AFG,1993,0.220987,4.976958,4.673549,0.700087,0.115089
4,Afghanistan,AFG,1994,0.220183,4.977782,4.670810,0.699898,0.111815
...,...,...,...,...,...,...,...,...
6415,Zimbabwe,ZWE,2015,0.201042,3.407624,3.184012,0.538596,0.095652
6416,Zimbabwe,ZWE,2016,0.201319,3.410755,3.187148,0.538593,0.096662
6417,Zimbabwe,ZWE,2017,0.201639,3.411965,3.188418,0.538589,0.097330
6418,Zimbabwe,ZWE,2018,0.201976,3.406929,3.172111,0.538585,0.097909


We want to simplify the names of the disorder columns and add a column that sums all the percentages to get the total disorder percent.

In [144]:
col = "disorders (share of population) - Sex: \
Both - Age: Age-standardized"
renamed_og_df = og_df.rename(columns={
    'Schizophrenia '+col:'Schizophrenia_percent',
    'Depressive '+col:'Depression_percent',
    'Anxiety '+col:'Anxiety_percent',
    'Bipolar '+col:'Bipolar_percent',
    'Eating '+col:'Eating_percent'})
renamed_og_df['Total_disorder_percent']=og_df.iloc[:,3:].sum(axis=1)

In [145]:
renamed_og_df.to_csv('clean_disorder_df.csv',index=False)

In [146]:
disorder_df = pd.read_csv('clean_disorder_df.csv')
disorder_df.head()

,Entity,Code,Year,Schizophrenia_percent,Depression_percent,Anxiety_percent,Bipolar_percent,Eating_percent,Total_disorder_percent
0,Afghanistan,AFG,1990,0.223206,4.996118,4.713314,0.703023,0.127700,10.763361
1,Afghanistan,AFG,1991,0.222454,4.989290,4.702100,0.702069,0.123256,10.739168
2,Afghanistan,AFG,1992,0.221751,4.981346,4.683743,0.700792,0.118844,10.706476
3,Afghanistan,AFG,1993,0.220987,4.976958,4.673549,0.700087,0.115089,10.686670
4,Afghanistan,AFG,1994,0.220183,4.977782,4.670810,0.699898,0.111815,10.680488


Load GDP dataframe:

In [147]:
gdp_df = pd.read_csv('source_gdp_data.csv')
gdp_df

,Entity,Code,Year,Depressive disorders (share of population) - Sex: Both - Age: Age-standardized,"GDP per capita, PPP (constant 2017 international $)",Continent
0,Abkhazia,OWID_ABK,2015,NaN,NaN,Asia
1,Afghanistan,AFG,1990,4.996118,NaN,NaN
2,Afghanistan,AFG,1991,4.989290,NaN,NaN
3,Afghanistan,AFG,1992,4.981346,NaN,NaN
4,Afghanistan,AFG,1993,4.976958,NaN,NaN
...,...,...,...,...,...,...
7293,Zimbabwe,ZWE,2017,3.411965,2331.7808,NaN
7294,Zimbabwe,ZWE,2018,3.406929,2399.6216,NaN
7295,Zimbabwe,ZWE,2019,3.395476,2203.3967,NaN
7296,Zimbabwe,ZWE,2020,NaN,1990.3195,NaN


Rename GDP per capita:

In [148]:
gdp_df.rename(columns={
    "GDP per capita, PPP (constant 2017 international $)":
'GDP_per_capita'},inplace=True)

Get rid of depressive disorder and continent column:

In [149]:
gdp_df = duckdb.sql("""SELECT Entity, Code, Year, GDP_per_capita
FROM gdp_df""").df()
gdp_df.to_csv("clean_gdp_df.csv",index=False)
gdp_df.head()
# We will deal with NaN values after combining all other 
# factors into main table 

,Entity,Code,Year,GDP_per_capita
0,Abkhazia,OWID_ABK,2015,NaN
1,Afghanistan,AFG,1990,NaN
2,Afghanistan,AFG,1991,NaN
3,Afghanistan,AFG,1992,NaN
4,Afghanistan,AFG,1993,NaN


Add GDP statistics to main disorder dataset:

In [150]:
disorders_gdp_df = disorder_df.merge(gdp_df)
disorders_gdp_df.head()

,Entity,Code,Year,Schizophrenia_percent,Depression_percent,Anxiety_percent,Bipolar_percent,Eating_percent,Total_disorder_percent,GDP_per_capita
0,Afghanistan,AFG,1990,0.223206,4.996118,4.713314,0.703023,0.127700,10.763361,NaN
1,Afghanistan,AFG,1991,0.222454,4.989290,4.702100,0.702069,0.123256,10.739168,NaN
2,Afghanistan,AFG,1992,0.221751,4.981346,4.683743,0.700792,0.118844,10.706476,NaN
3,Afghanistan,AFG,1993,0.220987,4.976958,4.673549,0.700087,0.115089,10.686670,NaN
4,Afghanistan,AFG,1994,0.220183,4.977782,4.670810,0.699898,0.111815,10.680488,NaN


Edited original population density csv file in Excel to get rid of missing lines and unnecessary columns that originally caused errors when reading into DataFrame.

In [151]:
pop_df = pd.read_csv('source_pop_data.csv')
pop_df

,Country Name,Country Code,1961,1962,1963,1964,1965,1966,1967,1968,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,310.061111,314.900000,319.305556,323.211111,326.566667,329.394444,330.677778,330.394444,...,567.288889,571.555556,575.522222,579.205556,582.633333,585.772222,588.677778,591.344444,592.138889,591.872222
1,Africa Eastern and Southern,AFE,9.207577,9.459187,9.719622,9.992443,10.276307,10.565442,10.864498,11.177568,...,37.219640,38.254520,39.316203,40.417928,41.520571,42.623255,43.769082,44.946930,46.150813,47.354183
2,Afghanistan,AFG,13.477056,13.751356,14.040239,14.343888,14.665298,14.999535,15.347393,15.711911,...,46.711251,48.359028,50.160542,51.750915,53.104284,54.648541,56.248231,57.908252,59.752281,61.480554
3,Africa Western and Central,AFW,10.978074,11.213633,11.459305,11.712711,11.975377,12.247555,12.526275,12.813873,...,41.654561,42.804993,43.982443,45.180225,46.405991,47.661860,48.934069,50.222984,51.536679,52.862913
4,Angola,AGO,4.364589,4.428812,4.491720,4.550573,4.601413,4.641890,4.674343,4.706989,...,20.203972,20.972970,21.760116,22.561740,23.385535,24.230872,25.085051,25.951382,26.813577,27.676084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,"Yemen, Rep.",YEM,10.695055,10.897184,11.099489,11.314664,11.548569,11.796939,12.061318,12.341428,...,49.668335,51.108968,52.566062,54.011677,55.446336,56.886545,58.318679,59.750916,61.147501,62.468779
263,South Africa,ZAF,14.005114,14.428553,14.872940,15.335298,15.816793,16.313522,16.825361,17.352826,...,43.809637,44.410238,45.115821,46.061301,46.511202,46.691679,47.267420,47.883549,48.472848,48.959480
264,Zambia,ZMB,4.330770,4.470637,4.615856,4.765687,4.920733,5.081693,5.247969,5.419999,...,19.834351,20.493921,21.170305,21.856939,22.555807,23.269151,23.992646,24.725214,25.461353,26.195032


Rename Country and Code columns to be consistent with main disorder dataset:

In [152]:
pop_df.rename(columns={"Country Name":"Entity",
                      "Country Code":"Code"},inplace=True)
pop_df.head()

,Entity,Code,1961,1962,1963,1964,1965,1966,1967,1968,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,310.061111,314.900000,319.305556,323.211111,326.566667,329.394444,330.677778,330.394444,...,567.288889,571.555556,575.522222,579.205556,582.633333,585.772222,588.677778,591.344444,592.138889,591.872222
1,Africa Eastern and Southern,AFE,9.207577,9.459187,9.719622,9.992443,10.276307,10.565442,10.864498,11.177568,...,37.219640,38.254520,39.316203,40.417928,41.520571,42.623255,43.769082,44.946930,46.150813,47.354183
2,Afghanistan,AFG,13.477056,13.751356,14.040239,14.343888,14.665298,14.999535,15.347393,15.711911,...,46.711251,48.359028,50.160542,51.750915,53.104284,54.648541,56.248231,57.908252,59.752281,61.480554
3,Africa Western and Central,AFW,10.978074,11.213633,11.459305,11.712711,11.975377,12.247555,12.526275,12.813873,...,41.654561,42.804993,43.982443,45.180225,46.405991,47.661860,48.934069,50.222984,51.536679,52.862913
4,Angola,AGO,4.364589,4.428812,4.491720,4.550573,4.601413,4.641890,4.674343,4.706989,...,20.203972,20.972970,21.760116,22.561740,23.385535,24.230872,25.085051,25.951382,26.813577,27.676084


Melt so all years are in one column:

In [153]:
tall_pop_df = pop_df.melt(id_vars=['Entity','Code'],
                          var_name='Year',
                          value_name='Population_density')
tall_pop_df

,Entity,Code,Year,Population_density
0,Aruba,ABW,1961,310.061111
1,Africa Eastern and Southern,AFE,1961,9.207577
2,Afghanistan,AFG,1961,13.477056
3,Africa Western and Central,AFW,1961,10.978074
4,Angola,AGO,1961,4.364589
...,...,...,...,...
16221,Kosovo,XKX,2021,NaN
16222,"Yemen, Rep.",YEM,2021,62.468779
16223,South Africa,ZAF,2021,48.959480
16224,Zambia,ZMB,2021,26.195032


`disorders_gdp_df` has dtype int for the Year column while `tall_pop_df` has dtype str for the Year column. We will convert the dtype of the Year column to int:

In [154]:
tall_pop_df['Year']=tall_pop_df['Year'].astype(int)
tall_pop_df.to_csv("clean_pop_df.csv",index=False)

Merge with main dataset:

In [155]:
disorders_gdp_pop_df = disorders_gdp_df.merge(tall_pop_df)
disorders_gdp_pop_df.head()

,Entity,Code,Year,Schizophrenia_percent,Depression_percent,Anxiety_percent,Bipolar_percent,Eating_percent,Total_disorder_percent,GDP_per_capita,Population_density
0,Afghanistan,AFG,1990,0.223206,4.996118,4.713314,0.703023,0.127700,10.763361,NaN,16.397277
1,Afghanistan,AFG,1991,0.222454,4.989290,4.702100,0.702069,0.123256,10.739168,NaN,16.474506
2,Afghanistan,AFG,1992,0.221751,4.981346,4.683743,0.700792,0.118844,10.706476,NaN,18.486474
3,Afghanistan,AFG,1993,0.220987,4.976958,4.673549,0.700087,0.115089,10.686670,NaN,21.470586
4,Afghanistan,AFG,1994,0.220183,4.977782,4.670810,0.699898,0.111815,10.680488,NaN,23.696480


Load political regime data into dataframe:

In [156]:
gov_df = pd.read_csv("source_gov_data.csv")
gov_df.head()

,Entity,Code,Year,regime_row_owid
0,Afghanistan,AFG,1789,0
1,Afghanistan,AFG,1790,0
2,Afghanistan,AFG,1791,0
3,Afghanistan,AFG,1792,0
4,Afghanistan,AFG,1793,0


Rename 'regime_row_owid':

In [157]:
gov_df.rename(
    columns={'regime_row_owid':'Political_regime_score'},
    inplace=True)
gov_df.head()

,Entity,Code,Year,Political_regime_score
0,Afghanistan,AFG,1789,0
1,Afghanistan,AFG,1790,0
2,Afghanistan,AFG,1791,0
3,Afghanistan,AFG,1792,0
4,Afghanistan,AFG,1793,0


Merge to main disorder dataset:

In [158]:
disorders_gdp_pop_gov_df = disorders_gdp_pop_df.merge(gov_df)
disorders_gdp_pop_gov_df.head()

,Entity,Code,Year,Schizophrenia_percent,Depression_percent,Anxiety_percent,Bipolar_percent,Eating_percent,Total_disorder_percent,GDP_per_capita,Population_density,Political_regime_score
0,Afghanistan,AFG,1990,0.223206,4.996118,4.713314,0.703023,0.127700,10.763361,NaN,16.397277,0
1,Afghanistan,AFG,1991,0.222454,4.989290,4.702100,0.702069,0.123256,10.739168,NaN,16.474506,0
2,Afghanistan,AFG,1992,0.221751,4.981346,4.683743,0.700792,0.118844,10.706476,NaN,18.486474,0
3,Afghanistan,AFG,1993,0.220987,4.976958,4.673549,0.700087,0.115089,10.686670,NaN,21.470586,0
4,Afghanistan,AFG,1994,0.220183,4.977782,4.670810,0.699898,0.111815,10.680488,NaN,23.696480,0


Load sunlight hour data into dataframe:

In [159]:
sun_hours_df = pd.read_csv('source_sunshine_data.csv')
sun_hours_df.head()

,Country,City,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
0,Afghanistan,Kabul,177.2,178.6,204.5,232.5,310.3,353.4,356.8,339.7,303.9,282.6,253.2,182.4,3175.1
1,Albania,Tirana,124.0,125.0,165.0,191.0,263.0,298.0,354.0,327.0,264.0,218.0,127.0,88.0,2544.0
2,Algeria,Algiers,149.0,165.0,202.0,258.0,319.0,318.0,350.0,319.0,237.0,229.0,165.0,136.0,2847.0
3,Algeria,Tamanrasset,297.6,275.5,322.4,327.0,328.6,306.0,356.5,331.7,288.0,310.0,285.0,272.8,3686.0
4,Angola,Luanda,219.0,208.0,213.0,199.0,233.0,223.0,175.0,150.0,145.0,164.0,199.0,212.0,2341.0


Use SQL to drop unnecessary columns and use the average of the annual sunlight hours of each Country's cities:

In [160]:
sun_hours_df = duckdb.sql("""SELECT Country AS Entity, 
                          AVG(Year) AS Annual_sunshine 
                          FROM sun_hours_df
                          GROUP BY Entity
                          ORDER BY Entity""").df()
sun_hours_df.head()

,Entity,Annual_sunshine
0,Afghanistan,3175.1
1,Albania,2544.0
2,Algeria,3266.5
3,Angola,2341.0
4,Argentina,2220.3


We will merge sun hours to main dataset after we have selected a year to focus on.

In [161]:
sun_hours_df.to_csv("clean_sun_hours.csv",index=False)

Get rid of rows with NaN values:

In [162]:
disorders_gdp_pop_gov_df.dropna(inplace=True,ignore_index=True)
disorders_gdp_pop_gov_df.to_csv('over_time_dataset.csv',index=False)
disorders_gdp_pop_gov_df.head()

,Entity,Code,Year,Schizophrenia_percent,Depression_percent,Anxiety_percent,Bipolar_percent,Eating_percent,Total_disorder_percent,GDP_per_capita,Population_density,Political_regime_score
0,Afghanistan,AFG,2002,0.214188,4.994327,4.672062,0.699779,0.094810,10.675167,1280.4631,32.197624,0
1,Afghanistan,AFG,2003,0.214093,4.981722,4.671881,0.699606,0.094983,10.662284,1292.3335,34.719547,0
2,Afghanistan,AFG,2005,0.214097,4.965879,4.673633,0.699439,0.095990,10.649038,1352.3207,37.427274,1
3,Afghanistan,AFG,2006,0.214259,4.962840,4.678040,0.699530,0.096875,10.651545,1366.9932,39.009159,1
4,Afghanistan,AFG,2007,0.214598,4.959720,4.682732,0.699520,0.098696,10.655267,1528.3446,39.714979,1


Choose most recent year for initial exploratory analysis:

In [163]:
df_2019 = duckdb.sql("""SELECT * FROM disorders_gdp_pop_gov_df
WHERE Year = 2019""").df().drop('Year',axis=1)
df_2019.head()

,Entity,Code,Schizophrenia_percent,Depression_percent,Anxiety_percent,Bipolar_percent,Eating_percent,Total_disorder_percent,GDP_per_capita,Population_density,Political_regime_score
0,Afghanistan,AFG,0.217777,4.945168,4.851035,0.699645,0.117414,10.831039,2079.9219,57.908252,1
1,Albania,ALB,0.287501,2.460203,3.837537,0.542009,0.136648,7.263898,13655.6650,104.167555,1
2,Algeria,DZA,0.249135,4.129681,4.771976,0.756275,0.210603,10.117671,11627.2800,17.930316,1
3,Angola,AGO,0.219828,5.744194,3.934095,0.553899,0.136756,10.588773,6602.4240,25.951382,1
4,Argentina,ARG,0.311542,2.407510,4.982973,1.039092,0.332201,9.073318,22071.7480,16.420827,2


Merge annual sunlight hours to the 2019 dataset:

In [164]:
complete_2019_df = df_2019.merge(sun_hours_df)

We decided to also look at pollution levels per country, so below is the cleaning of CO<sub>2</sub> emissions data:

In [165]:
co2 = pd.read_csv('source_co2_data.csv')
co2.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,1.001154,1.013758,0.960430,0.941337,0.933874,0.921453,0.915294,0.795420,NaN,NaN
2,Afghanistan,AFG,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,0.298088,0.283692,0.297972,0.268359,0.281196,0.299083,0.297564,0.223479,NaN,NaN
3,Africa Western and Central,AFW,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,0.481623,0.493505,0.475577,0.479775,0.465166,0.475817,0.490837,0.463150,NaN,NaN
4,Angola,AGO,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,1.031044,1.091497,1.125185,1.012552,0.829723,0.755828,0.753638,0.592743,NaN,NaN


Rename relevant columns:

In [166]:
renamed = co2.rename(columns={"Country Name":'Entity',
                              "Country Code":'Code',
                              '2019':'CO2_emissions'})
renamed.to_csv("clean_pollution.csv",index=False)

Merge with main dataframe:

In [167]:
co2 = duckdb.sql("""SELECT Entity, Code, CO2_emissions
FROM renamed""").df()
with_pollution = complete_2019_df.merge(co2)
with_pollution

,Entity,Code,Schizophrenia_percent,Depression_percent,Anxiety_percent,Bipolar_percent,Eating_percent,Total_disorder_percent,GDP_per_capita,Population_density,Political_regime_score,Annual_sunshine,CO2_emissions
0,Afghanistan,AFG,0.217777,4.945168,4.851035,0.699645,0.117414,10.831039,2079.9219,57.908252,1,3175.100000,0.297564
1,Albania,ALB,0.287501,2.460203,3.837537,0.542009,0.136648,7.263898,13655.6650,104.167555,1,2544.000000,1.749462
2,Algeria,DZA,0.249135,4.129681,4.771976,0.756275,0.210603,10.117671,11627.2800,17.930316,1,3266.500000,3.994402
3,Angola,AGO,0.219828,5.744194,3.934095,0.553899,0.136756,10.588773,6602.4240,25.951382,1,2341.000000,0.753638
4,Argentina,ARG,0.311542,2.407510,4.982973,1.039092,0.332201,9.073318,22071.7480,16.420827,2,2220.300000,3.742030
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,United States,USA,0.433649,4.375998,5.697748,0.601298,0.424350,11.533043,62478.2540,35.893176,3,2827.618519,14.673381
112,Uruguay,URY,0.313488,2.840908,5.153944,0.984342,0.353573,9.646255,23256.5980,19.588670,3,2481.400000,1.985382
113,Uzbekistan,UZB,0.270145,3.164448,2.025655,0.511308,0.115607,6.087163,7348.1470,76.222833,1,2823.900000,3.504663
114,Zambia,ZMB,0.212869,4.219490,3.969207,0.574454,0.114669,9.090688,3372.3590,24.725214,1,2965.466667,0.414336


In [168]:
with_pollution.to_csv('final_dataset.csv',index=False)